In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [8]:
preprocess_input = tf.keras.applications.resnet50.preprocess_input

train_ds = tf.keras.utils.image_dataset_from_directory(
    "brain_tumour_classification/Training",
    image_size=(224, 224),
    batch_size=32,
    color_mode='rgb',
    label_mode='categorical'
)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    "brain_tumour_classification/Testing",
    image_size=(224 , 224),
    batch_size=32,
    color_mode='rgb',
    label_mode='categorical'
)

Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.


In [3]:
for images, labels in train_ds.take(1):
    print(images.shape)

(32, 224, 224, 3)


In [4]:
class_names = train_ds.class_names
print(f"Detected Classes: {class_names}")

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    '../models/brain_tumor.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

Detected Classes: ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']


In [6]:
base_model = tf.keras.applications.ResNet50(
    include_top=False, 
    weights='imagenet',
    input_shape=(224,224,3)
)

#Freezing the brain - so the model doesn't learn anything new; instead uses its own learnt knowledge
base_model.trainable = False

# custom layers
model = models.Sequential([

    layers.Lambda(preprocess_input, input_shape=(224,224,3)),

    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),

    base_model,

    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(4, activation='softmax')
])

model.compile(
    optimizer = 'adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("PHASE 1: Training the new layers (warm up) ...")

history_warmup = model.fit(
    train_ds,
    epochs=10,
    validation_data=validation_ds,
    callbacks=[checkpoint]
)

# --- Fine Tuning 
print("\n Phase 2: Unfreezing Block 5 for the fine-tuning")

base_model.trainable = True

fine_tune_at =len(base_model.layers) - 20

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
    
# Re-Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_finetune = model.fit(
    train_ds,
    epochs=10,
    validation_data=validation_ds,
    callbacks=[checkpoint, early_stopping]
)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


d:\Downloads\Necessities\Anaconda_Install\envs\ai_bot\lib\site-packages\keras\src\layers\core\lambda_layer.py:65: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



PHASE 1: Training the new layers (warm up) ...
Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6300 - loss: 1.0239
Epoch 1: val_accuracy improved from None to 0.52538, saving model to ../models/brain_tumor.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - accuracy: 0.7216 - loss: 0.7454 - val_accuracy: 0.5254 - val_loss: 1.2493
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8237 - loss: 0.4544
Epoch 2: val_accuracy improved from 0.52538 to 0.59137, saving model to ../models/brain_tumor.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.8150 - loss: 0.4676 - val_accuracy: 0.5914 - val_loss: 1.1881
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8671 - loss: 0.3748
Epoch 3: val_accuracy improved from 0.59137 to 0.63959, saving model to ../models/brain_tumor.h5


...

90/90 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.8958 - loss: 0.2730 - val_accuracy: 0.7081 - val_loss: 1.3190

 Phase 2: Unfreezing Block 5 for the fine-tuning
Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8638 - loss: 0.3466
Epoch 1: val_accuracy did not improve from 0.70812
90/90 ━━━━━━━━━━━━━━━━━━━━ 191s 2s/step - accuracy: 0.8777 - loss: 0.3103 - val_accuracy: 0.6980 - val_loss: 1.3985
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9219 - loss: 0.2148
Epoch 2: val_accuracy did not improve from 0.70812
90/90 ━━━━━━━━━━━━━━━━━━━━ 180s 2s/step - accuracy: 0.9181 - loss: 0.2203 - val_accuracy: 0.7081 - val_loss: 1.4526
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9230 - loss: 0.2113
Epoch 3: val_accuracy improved from 0.70812 to 0.73096, saving model to ../models/brain_tumor.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 174s 2s/step - accuracy: 0.9268 - loss: 0.2063 - val_accuracy: 0.7310 - val_loss: 1.4991
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9411 - loss: 0.1783
Epoch 4: val_accuracy improved from 0.73096 to 0.74112, saving model to ../models/brain_tumor.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step - accuracy: 0.9404 - loss: 0.1785 - val_accuracy: 0.7411 - val_loss: 1.5323
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9442 - loss: 0.1582
Epoch 5: val_accuracy did not improve from 0.74112
90/90 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step - accuracy: 0.9380 - loss: 0.1716 - val_accuracy: 0.7386 - val_loss: 1.6135
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9366 - loss: 0.1631
Epoch 6: val_accuracy improved from 0.74112 to 0.74619, saving model to ../models/brain_tumor.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 183s 2s/step - accuracy: 0.9380 - loss: 0.1579 - val_accuracy: 0.7462 - val_loss: 1.6016
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9562 - loss: 0.1293
Epoch 7: val_accuracy improved from 0.74619 to 0.74873, saving model to ../models/brain_tumor.h5


90/90 ━━━━━━━━━━━━━━━━━━━━ 172s 2s/step - accuracy: 0.9523 - loss: 0.1341 - val_accuracy: 0.7487 - val_loss: 1.6805
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9546 - loss: 0.1232
Epoch 8: val_accuracy did not improve from 0.74873
90/90 ━━━━━━━━━━━━━━━━━━━━ 175s 2s/step - accuracy: 0.9561 - loss: 0.1233 - val_accuracy: 0.7462 - val_loss: 1.6269
Epoch 9/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9589 - loss: 0.1128
Epoch 9: val_accuracy did not improve from 0.74873
90/90 ━━━━━━━━━━━━━━━━━━━━ 175s 2s/step - accuracy: 0.9627 - loss: 0.1100 - val_accuracy: 0.7487 - val_loss: 1.6291
Epoch 10/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9596 - loss: 0.1091
Epoch 10: val_accuracy did not improve from 0.74873
90/90 ━━━━━━━━━━━━━━━━━━━━ 173s 2s/step - accuracy: 0.9620 - loss: 0.1100 - val_accuracy: 0.7487 - val_loss: 1.6689
